<a href="https://colab.research.google.com/github/adheeb4/7PAM2015-0105-2023-Individual-Project/blob/main/Individual_Project_Image_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install fiftyone
!pip install fiftyone-db==0.4.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.9/108.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 50.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.5/192.5 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.6/89.6 kB 8.3 MB/s eta 0:00:00
  Installing build dependenci

In [ ]:
!pip install albumentations

In [4]:
import fiftyone as fo
import fiftyone.zoo as foz
import numpy as np
import cv2
import matplotlib.pyplot as plt

Migrating database to v0.23.6


INFO:fiftyone.migrations.runner:Migrating database to v0.23.6


In [ ]:
!fiftyone plugins download https://github.com/jacobmarks/fiftyone-albumentations-plugin.git



Copying plugin '@jacobmarks/albumentations_augmentation' to '/root/fiftyone/__plugins__/@jacobmarks/albumentations_augmentation'


In [9]:

dataset_dir = "/content/drive/MyDrive/data"

train_images_dir = "/content/drive/MyDrive/data/validation-300/data"
train_labels_dir = "/content/drive/MyDrive/data/validation-300/labels.json"
validation_images_dir = "/content/drive/MyDrive/data/validation-300/data"
validation_labels_dir ="/content/drive/MyDrive/data/validation-300/labels.json"
test_images_dir = "/content/drive/MyDrive/data/test-30"


dataset_type = fo.types.COCODetectionDataset


"""
train_dataset = foz.load_zoo_dataset(
    "coco-2017",
    split="train",
    max_samples=50,
    shuffle=True,
)
"""
train_dataset = fo.Dataset.from_dir(
    data_path=train_images_dir,
    labels_path=train_labels_dir,
    label_field="ground_truth",
    label_types=["detections", "segmentations"],
    name="train1",
    dataset_type = dataset_type,
    include_id=True
)

validation_dataset = fo.Dataset.from_dir(
    data_path=validation_images_dir,
    labels_path=validation_labels_dir,
    label_field="ground_truth",
    label_types=["detections", "segmentations"],
    name="validation1",
    dataset_type = dataset_type,
    include_id=True
)


# Load the test images into a FiftyOne dataset (no annotations)
test_dataset = fo.Dataset.from_dir(
    dataset_dir=test_images_dir,
    name="test1",
    dataset_type=fo.types.ImageDirectory,
    include_id=True
)


 100% |█████████████████| 300/300 [13.7s elapsed, 0s remaining, 31.4 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 300/300 [13.7s elapsed, 0s remaining, 31.4 samples/s]      


 100% |█████████████████| 300/300 [12.9s elapsed, 0s remaining, 27.7 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 300/300 [12.9s elapsed, 0s remaining, 27.7 samples/s]      


Ignoring unsupported parameter 'include_id' for importer type <class 'fiftyone.utils.data.importers.ImageDirectoryImporter'>


 100% |███████████████████| 30/30 [40.7ms elapsed, 0s remaining, 736.6 samples/s]     


INFO:eta.core.utils: 100% |███████████████████| 30/30 [40.7ms elapsed, 0s remaining, 736.6 samples/s]     


In [10]:
print(train_dataset)
print(validation_dataset)

Name:        train1
Media type:  image
Num samples: 300
Persistent:  False
Tags:        []
Sample fields:
    id:                         fiftyone.core.fields.ObjectIdField
    filepath:                   fiftyone.core.fields.StringField
    tags:                       fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:                   fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth_detections:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    ground_truth_segmentations: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    ground_truth_coco_id:       fiftyone.core.fields.IntField
Name:        validation1
Media type:  image
Num samples: 300
Persistent:  False
Tags:        []
Sample fields:
    id:                         fiftyone.core.fields.ObjectIdField
    filepath:                   fiftyone.core.fields.StringField
    tags:                      

In [13]:
import fiftyone as fo
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

# Load FiftyOne dataset
# dataset = fo.load_dataset("your_dataset_name")

# Convert DatasetView to list and sample from it
train_dataset_list = list(train_dataset)
dataset = train_dataset_list[:100]  # Sample a subset of your dataset

# Preprocess data (resize, normalize, etc.)
# Example: Resize images to (224, 224)
images = np.array([fo.io.read_image(sample['filepath']) for sample in dataset])
#images = np.array([fo.utils.resize(img, (224, 224)) for img in images])  # Resize images
#images = images.astype('float32') / 255.0  # Normalize images

# Get labels (assuming you have labels in your dataset)
labels = np.array([sample['ground_truth_segmentations'].label for sample in train_dataset])

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

# Define Keras model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')  # Adjust output units based on your dataset
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))



AttributeError: module 'fiftyone' has no attribute 'io'

In [20]:
import fiftyone as fo

# Load the dataset
dataset = fo.Dataset

# Assuming you have a sample object
sample = train_dataset.first()

# Access the ground truth segmentations
ground_truth_detections = sample.ground_truth_detections

# Assuming there is only one segmentation in the sample
detection = ground_truth_detections.detections[0]

# Access the label associated with the segmentation
label = detection.label
mask = detection.mask


# Print the label
print(label)
print(mask)

stop sign
None


In [22]:
print(sample.ground_truth_detections)


<Detections: {
    'detections': [
        <Detection: {
            'id': '65f31d525938e05b0c593c17',
            'attributes': {},
            'tags': [],
            'label': 'stop sign',
            'bounding_box': [
                0.3201866666666667,
                0.14366,
                0.35864,
                0.30616000000000004,
            ],
            'mask': None,
            'confidence': None,
            'index': None,
            'supercategory': 'outdoor',
            'iscrowd': 0,
        }>,
        <Detection: {
            'id': '65f31d525938e05b0c593c18',
            'attributes': {},
            'tags': [],
            'label': 'truck',
            'bounding_box': [
                0.32904,
                0.55888,
                0.05797333333333333,
                0.060340000000000005,
            ],
            'mask': None,
            'confidence': None,
            'index': None,
            'supercategory': 'vehicle',
            'iscrowd': 0,
     

In [40]:
print(sample.ground_truth_detections.detection.bounding_box)

AttributeError: 'Detections' object has no attribute 'detection'

In [41]:
# Assuming `sample` contains a `Detections` object
detections = sample.ground_truth_detections.detections

# Extract bounding box coordinates for each detection
bounding_boxes = []

for detection in detections:
    bounding_box = detection.bounding_box
    bounding_boxes.append(bounding_box)

print("Bounding Boxes:", bounding_boxes)

Bounding Boxes: [[0.3201866666666667, 0.14366, 0.35864, 0.30616000000000004], [0.32904, 0.55888, 0.05797333333333333, 0.060340000000000005], [0.34202666666666665, 0.53442, 0.03434666666666667, 0.016300000000000002], [0.5396799999999999, 0.5197999999999999, 0.05069333333333334, 0.05212]]


In [45]:
print(detection['bounding_box'])

[0.5396799999999999, 0.5197999999999999, 0.05069333333333334, 0.05212]


In [55]:
import numpy as np
import cv2

def create_mask(image_shape, bounding_box):
    mask = np.zeros(image_shape[:2], dtype=np.uint8)
    x_min, y_min, x_max, y_max = bounding_box
    mask[y_min:y_max, x_min:x_max] = 255
    return mask

# Assuming you have the image shape (height, width) and the detections list
# image_shape = (height, width)  # Provide the actual height and width of your images
masks = []

for detection in sample.ground_truth_detections:
    metadata = sample.metadata
    image_shape = (metadata.width, metadata.height, metadata.num_channels)
    detection_list = detection.split(',')  # Split the string into a list
    bounding_box = [int(coord) for coord in detection_list]  # Convert string coordinates to integers
    mask = create_mask(image_shape, bounding_box)
    masks.append(mask)

# You now have a list of masks corresponding to each detection.


ValueError: invalid literal for int() with base 10: 'detections'

In [59]:
import numpy as np
import cv2
detections = sample.ground_truth_detections.detections

def create_mask(image_shape, bounding_box):
    mask = np.zeros(image_shape[:2], dtype=np.uint8)
    x, y, w, h = bounding_box
    mask = [x, y, w, h]
    return mask

# Assuming you have the image shape (height, width) and the detections list
#image_shape = (height, width)  # Provide the actual height and width of your images
masks = []

for detection in detections:
    image_shape = (metadata.width, metadata.height, metadata.num_channels)
    mask = create_mask(image_shape, detection['bounding_box'])
    masks.append(mask)
    print(detection['bounding_box'])
# You now have a list of masks corresponding to each detection
print(masks)

[0.3201866666666667, 0.14366, 0.35864, 0.30616000000000004]
[0.32904, 0.55888, 0.05797333333333333, 0.060340000000000005]
[0.34202666666666665, 0.53442, 0.03434666666666667, 0.016300000000000002]
[0.5396799999999999, 0.5197999999999999, 0.05069333333333334, 0.05212]
[[0.3201866666666667, 0.14366, 0.35864, 0.30616000000000004], [0.32904, 0.55888, 0.05797333333333333, 0.060340000000000005], [0.34202666666666665, 0.53442, 0.03434666666666667, 0.016300000000000002], [0.5396799999999999, 0.5197999999999999, 0.05069333333333334, 0.05212]]


In [ ]:
!pip install voxel51-utils

ERROR: Could not find a version that satisfies the requirement voxel51-utils (from versions: none)
ERROR: No matching distribution found for voxel51-utils


In [ ]:
import fiftyone.operators as foo
print(train_dataset)
compute_metadata = foo.get_operator("@voxel51/utils/compute_metadata")

import asyncio
import fiftyone as fo

# Define a function to compute metadata
async def compute_metadata_async(dataset):
    await dataset.compute_metadata(overwrite=True, delegate=True)

# Load your dataset
dataset = fo.Dataset.from_name("train2")

# Run the function within asyncio.run()
asyncio.run(compute_metadata_async(dataset))

Name:        train
Media type:  image
Num samples: 300
Persistent:  False
Tags:        []
Sample fields:
    id:            fiftyone.core.fields.ObjectIdField
    filepath:      fiftyone.core.fields.StringField
    tags:          fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    segmentations: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    coco_id:       fiftyone.core.fields.IntField


ValueError: Operator '@voxel51/utils/compute_metadata' not found

In [ ]:
session = fo.launch_app(train_dataset)


Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.23.6

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



INFO:fiftyone.core.session.session:
Welcome to

███████╗██╗███████╗████████╗██╗   ██╗ ██████╗ ███╗   ██╗███████╗
██╔════╝██║██╔════╝╚══██╔══╝╚██╗ ██╔╝██╔═══██╗████╗  ██║██╔════╝
█████╗  ██║█████╗     ██║    ╚████╔╝ ██║   ██║██╔██╗ ██║█████╗
██╔══╝  ██║██╔══╝     ██║     ╚██╔╝  ██║   ██║██║╚██╗██║██╔══╝
██║     ██║██║        ██║      ██║   ╚██████╔╝██║ ╚████║███████╗
╚═╝     ╚═╝╚═╝        ╚═╝      ╚═╝    ╚═════╝ ╚═╝  ╚═══╝╚══════╝ v0.23.6

If you're finding FiftyOne helpful, here's how you can get involved:

|
|  ⭐⭐⭐ Give the project a star on GitHub ⭐⭐⭐
|  https://github.com/voxel51/fiftyone
|
|  🚀🚀🚀 Join the FiftyOne Slack community 🚀🚀🚀
|  https://slack.voxel51.com
|



In [ ]:
print(train_dataset.summary())
train_dataset.compute_metadata()

Name:        coco-2017-train-50
Media type:  image
Num samples: 50
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)


In [ ]:

for sample in train_dataset:
    num_detections = len(sample.detections.detections)
    sample["metadata"]["num_detections"] = num_detections

# Compute object classes distribution
class_distribution = {}
for sample in train_dataset:
    for detection in sample.detections.detections:
        label = detection.label
        class_distribution[label] = class_distribution.get(label, 0) + 1

# Compute object sizes statistics
total_area = 0
total_detections = 0
for sample in train_dataset:
    for detection in sample.detections.detections:
        bbox = detection.bounding_box
        # Bounding box format: [x_min, y_min, width, height]
        width = bbox[2]
        height = bbox[3]
        area = width * height
        total_area += area
        total_detections += 1

average_area = total_area / total_detections
average_area = total_area / total_detections


Could not connect session, trying again in 10 seconds



In [ ]:
print(class_distribution)
print(average_area)

NameError: name 'class_distribution' is not defined

In [ ]:
resized_train_dataset = fo.Dataset(name="resized_train_dataset")
resized_validation_dataset = fo.Dataset(name="resized_validation_dataset")

# Define the width and height you want to resize the images to
width = 256
height = 256
def resize(width, height, dataset, resized_dataset):
# Iterate over the samples in the dataset and resize the images
  for sample in dataset:
    image_path = sample.filepath
    image = cv2.imread(image_path)
    resized_image = cv2.resize(image, (width, height))

    # Update the sample with the resized image
    sample["image"] = resized_image

    # Update labels (if any)
    #for label in sample.labels.values():
        # Perform any necessary updates to the label coordinates or attributes
        # based on the resized image dimensions

    # Update metadata (if any)
    sample.metadata["resized"] = True
    sample.metadata["original_width"] = sample.metadata.width
    sample.metadata["original_height"] = sample.metadata.height
    sample.metadata.width = width
    sample.metadata.height = height

    # Add the sample to the resized dataset
    resized_dataset.add_sample(sample)
  return resized_dataset



In [ ]:
resized_train_dataset = resize(256, 256, train_dataset, resized_train_dataset)
resized_validation_dataset = resize(256, 256, validation_dataset, resized_validation_dataset)

In [ ]:
print(resized_train_dataset)
print(resized_validation_dataset)

Name:        resized_train_dataset
Media type:  image
Num samples: 50
Persistent:  False
Tags:        []
Sample fields:
    id:           fiftyone.core.fields.ObjectIdField
    filepath:     fiftyone.core.fields.StringField
    tags:         fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth: fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    image:        fiftyone.core.fields.ArrayField
Name:        resized_validation_dataset
Media type:  image
Num samples: 300
Persistent:  False
Tags:        []
Sample fields:
    id:            fiftyone.core.fields.ObjectIdField
    filepath:      fiftyone.core.fields.StringField
    tags:          fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    detections:    fiftyone.core.f

In [ ]:
session = fo.launch_app()

In [ ]:
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 90.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.1 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py

In [ ]:
print(train_dataset.sample)

AttributeError: 'Dataset' object has no attribute 'sample'

In [ ]:
print(resized_train_dataset)
print

NameError: name 'resized_train_dataset' is not defined

In [ ]:
import json
import numpy as np

# Assuming `resized_train_dataset` is your dataset
y_train = []

for sample in resized_train_dataset:
    segmentation = sample.ground_truth_segmentations
    if segmentation is not None:
        segmentation_masks = []
        for seg in segmentation:
            # Parse the string segmentation mask into a numpy array
            mask_data = json.loads(seg)
            mask = np.array(mask_data["mask"])
            segmentation_masks.append(mask)
        y_train.append(segmentation_masks)
    else:
        # If no ground truth segmentation is available, append None
        y_train.append(None)

y_train = np.array(y_train)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
from fiftyone.core.evaluation import ClassificationResults

# Assuming `results` is your classification evaluation results object
results.print_report(classes=classes)

# Get the 10 most common classes in the dataset
counts = train_dataset.count_values("ground_truth_detections.label")
classes = sorted(counts, key=counts.get, reverse=True)[:10]

# Print a classification report for the top-10 classes
train_dataset.print_classification_report(classes=classes)

ImportError: cannot import name 'ClassificationResults' from 'fiftyone.core.evaluation' (/usr/local/lib/python3.10/dist-packages/fiftyone/core/evaluation.py)

In [ ]:
import fiftyone as fo

# Sample information
sample_id = '65ee036cbcc5f296d232f789'

# Retrieve the sample by its ID
sample = resized_train_dataset.match({"id": sample_id}).first()

# Retrieve segmentation masks
segmentations = sample.segmentations

# Initialize a list to store segmentation masks
segmentation_masks = []

# Iterate over segmentations
for detection in segmentations.detections:
    # Check if the detection has a mask
    if detection.mask is not None:
        segmentation_masks.append(detection.mask)

# Now `segmentation_masks` contains all segmentation masks for the sample
print("Segmentation masks:", segmentation_masks)

ValueError: DatasetView is empty

In [ ]:
import fiftyone as fo

# Sample information
sample_id = '65ee036cbcc5f296d232f789'

# Retrieve the sample by its ID
sample = resized_train_dataset.match({"id": sample_id}).first()

# Retrieve segmentation masks
segmentations = sample.segmentations

# Initialize a list to store segmentation masks
segmentation_masks = []

# Iterate over segmentations
for detection in segmentations.detections:
    # Check if the detection has a mask
    if detection.mask is not None:
        segmentation_masks.append(detection.mask)

# Now `segmentation_masks` contains all segmentation masks for the sample
print("Segmentation masks:", segmentation_masks)


ValueError: DatasetView is empty

In [ ]:
import fiftyone as fo


y_train = []
for sample in resized_train_dataset:
    # Retrieve segmentation masks
    segmentation_masks = sample.segmentations

    # Initialize list to store segmentation masks for current sample
    segmentation_masks_list = []

    # Iterate over segmentation masks
    for segmentation_mask in segmentation_masks:
        segmentation_masks_list.append(segmentation_mask.mask)

    # Append segmentation masks list to y_train
    y_train.append(segmentation_masks_list)

AttributeError: Sample has no field 'segmentations'

In [ ]:
!pip install --upgrade tensorflow


In [ ]:
import autokeras as ak



# Prepare your data for training

x_train = []
y_train = []
x_val = []
y_val = []

for sample in resized_train_dataset:
    # Retrieve input image
    input_image = sample["image"]
    x_train.append(input_image)

    # Retrieve segmentation mask
    segmentation_masks = sample.segmentations
    segmentation_mask = segmentation_masks[0].mask if segmentation_masks else None
    y_train.append(segmentation_mask)

for sample in resized_validation_dataset:
    # Retrieve input image
    input_image = sample["image"]
    x_val.append(input_image)

    # Retrieve segmentation mask
    segmentation_mask = sample["segmentations"]
    y_val.append(segmentation_mask)

# Convert lists to numpy arrays if necessary
import numpy as np
x_train = np.array(x_train)
y_train = np.array(y_train)

x_val = np.array(x_val)
y_val = np.array(y_val)



ModuleNotFoundError: No module named 'tensorflow.keras.layers.experimental'

In [ ]:
import numpy as np

# Assuming dataset is your dataset object containing detection data
for sample in train_dataset:
    # Accessing the detections from the sample
    detections = sample.detections

    # Iterate over each detection in the sample
    for detection in detections.detections:
        label = detection.label
        mask = detection.mask

        # Flatten the mask into a 1D array
        flattened_mask = np.ravel(mask)

        # Print or store the flattened mask and its corresponding label for ML training
        print("Label:", label)
        print("Flattened Mask:", flattened_mask)


AttributeError: Sample has no field 'detections'

In [ ]:
print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)

print("x_val shape:", x_val.shape)
print("y_val shape:", y_val.shape)

x_train shape: (300, 256, 256, 3)
y_train shape: (300, 2)
x_val shape: (300, 256, 256, 3)
y_val shape: (300, 2)


In [ ]:
print(y_train)

NameError: name 'y_train' is not defined

In [ ]:
import numpy as np

# Convert categorical labels to segmentation masks
num_classes = 2  # Number of classes (assuming binary segmentation)
height, width = 256, 256  # Image dimensions

y_train_masks = np.zeros((len(y_train), height, width, num_classes), dtype=np.uint8)
y_val_masks = np.zeros((len(y_val), height, width, num_classes), dtype=np.uint8)

for i in range(len(y_train)):
    label_index = int(y_train[i][0])  # Convert to integer
    y_train_masks[i, :, :, label_index] = 1

for i in range(len(y_val)):
    label_index = int(y_val[i][0])  # Convert to integer
    y_val_masks[i, :, :, label_index] = 1

# Now, y_train_masks and y_val_masks contain segmentation masks


ValueError: invalid literal for int() with base 10: 'detections'

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

# Define the U-Net architecture for segmentation
def unet_segmentation(input_shape, num_classes):
    inputs = Input(input_shape)

    # Encoder
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    # Decoder
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
    up1 = UpSampling2D(size=(2, 2))(conv2)

    # Output layer
    outputs = Conv2D(num_classes, 1, activation='softmax')(up1)

    model = Model(inputs=inputs, outputs=outputs)
    return model

# Create the segmentation model
input_shape = (256, 256, 3)  # Example input shape, adjust as needed
num_classes = 2  # Example number of classes (background + foreground)
model = unet_segmentation(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=10)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1155, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1249, in compute_metrics
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/compile_utils.py", line 620, in update_state
        metric_obj

In [ ]:
auto_model = ak.AutoModel(
    inputs=ak.ImageInput(),
    outputs="segmentation",
    max_trials=10,
)
auto_model.fit(x_train, y_train, validation_data=(x_val, y_val))
"""
# Evaluate the trained model on your test dataset
# (Assuming you have test data for evaluation)
x_test = ...  # Extract test images
y_test = ...  # Extract ground truth segmentation masks
results = auto_model.evaluate(x_test, y_test)

# Visualize the results in FiftyOne
session = fo.launch_app(dataset)
session.view(dataset=dataset, eval_key="auto_ml_results")
"""

UnboundLocalError: local variable 'graph' referenced before assignment

In [ ]:

# Define and train an AutoKeras image segmentation model
clf = ak.SegmentationHead()
auto_model = ak.AutoModel(inputs=ak.ImageInput(), outputs=clf, max_trials=10)
auto_model.fit(x_train, y_train)

# Evaluate the trained model on your dataset
# (Assuming you have test data for evaluation)
x_test = ...  # Extract test images
y_test = ...  # Extract ground truth segmentation masks
results = auto_model.evaluate(x_test, y_test)

# Visualize the results in FiftyOne
session = fo.launch_app(dataset)
session.view(dataset=dataset, eval_key="auto_ml_results")

In [ ]:
num_columns = 4

# Calculate the number of rows required based on the number of samples and number of columns
num_samples = len(resized_dataset)
num_rows = (num_samples + num_columns - 1) // num_columns

# Create a subplot grid
fig, axes = plt.subplots(num_rows, num_columns, figsize=(15, 5 * num_rows))

# Iterate over the samples in the dataset and visualize the images
for i, sample in enumerate(resized_dataset):
    # Calculate the row and column index for the subplot
    row_index = i // num_columns
    col_index = i % num_columns

    # Load and display the image
    image = sample["image"]
    axes[row_index, col_index].imshow(image)
    axes[row_index, col_index].axis("off")

    # Set the title as the sample ID
    axes[row_index, col_index].set_title(sample.id)

# Adjust layout and display the plot
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Access file paths of images
filepaths = [sample.filepath for sample in train_dataset]

# Get image dimensions
image_sizes = []
for filepath in filepaths:
    # Load image and get dimensions
    image = plt.imread(filepath)
    if len(image.shape) == 2:  # Grayscale image
        height, width = image.shape
    else:  # Color image
        height, width, _ = image.shape
    image_sizes.append((width, height))

# Convert to numpy array for easier manipulation
image_sizes = np.array(image_sizes)

# Plot KDE plot
plt.figure(figsize=(10, 6))
sns.kdeplot(data=image_sizes[:, 0], label="Width", fill=True)
sns.kdeplot(data=image_sizes[:, 1], label="Height", fill=True)

# Set plot labels and title
plt.xlabel("Image Dimension")
plt.ylabel("Density")
plt.title("Kernel Density Estimate (KDE) of Image Size Distribution")

# Add legend
plt.legend()

# Show plot
plt.show()


In [ ]:


def preprocess(sample, resize_shape=(256, 256)):
  image_data = cv2.imread(sample.filepath)
  binary_mask = np.zeros(image_data.shape[:2], dtype=np.uint8)

  detections = sample["detections"]
  if detections is not None:
    for detection in detections.values():
      for seg in detection.segmentation:
        cv2.fillPoly(binary_mask, [np.array(seg)], 255)
  #for seg in sample.mask.ground_truth.segmentation:
   # cv2.fillPoly(binary_mask, np.array(seg), 255)
  if resize_shape:
    image_data = cv2.resize(image_data, resize_shape)
    binary_mask = cv2.resize(binary_mask, resize_shape, interpolation=cv2.INTER_NEAREST)

  image_data = image_data / 255.0

  return image_data, binary_mask


preprocessed_images = []
preprocessed_masks = []

for sample in dataset:
 # image = sample["image"]
#mask = sample["ground_truth"]

  preprocessed_image, preprocessed_mask = preprocess(sample)

  preprocessed_images.append(preprocessed_image)
  preprocessed_masks.append(preprocessed_mask)
